In [2]:
import ee
import osmnx as ox
import geopandas as gpd
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from shapely.geometry import Polygon, MultiPolygon
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# ee.Authenticate()
project_id = 'bengaluru-lakes-485612'

try:
    ee.Initialize(project=project_id)
    print("Successfully initialized!")
except Exception:
    ee.Authenticate()
    ee.Initialize(project=project_id)

Successfully initialized!


In [4]:
# Define Bengaluru boundary
place_name = 'Bengaluru, Karnataka, India'

# Query lakes using tags: natural=water and water=lake
tags = {'natural': 'water', 'water': 'lake'}

print(f'Searching for lakes in {place_name}...')
try:
    # Use osmnx to fetch features
    lakes_gdf = ox.features_from_place(place_name, tags)
    
    # Filter for polygons and multipolygons
    lakes_gdf = lakes_gdf[lakes_gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]
    
    # Keep only relevant columns and drop rows without names
    lakes_gdf = lakes_gdf[['name', 'geometry']].dropna(subset=['name'])
    
    print(f'Found {len(lakes_gdf)} lakes with names.')
    display(lakes_gdf.head())
except Exception as e:
    print(f'Error retrieving lakes: {e}')


Searching for lakes in Bengaluru, Karnataka, India...
Found 202 lakes with names.


name  \
element  id                         
relation 1332093        NCBS Pond   
         1853330  Vengayyana Lake   
         1857615    Halasuru lake   
         2310400        Chelekere   
         2310417    Madiwala Lake   

                                                           geometry  
element  id                                                          
relation 1332093  POLYGON ((77.5791 13.07125, 77.57909 13.07121,...  
         1853330  POLYGON ((77.70218 13.01708, 77.70235 13.017, ...  
         1857615  POLYGON ((77.62261 12.98202, 77.6227 12.98193,...  
         2310400  POLYGON ((77.64527 13.02519, 77.64512 13.02543...  
         2310417  MULTIPOLYGON (((77.61159 12.90261, 77.61165 12...

In [ ]:
import geemap
import geopandas as gpd

# 1. Calculate the centroid correctly using projection
# We use a temporary object so we don't clutter the GeoDataFrame with objects
temp_centroids = lakes_gdf.to_crs(epsg=32643).centroid.to_crs(epsg=4326)

# 2. Store only the numbers (floats), which are JSON-friendly
lakes_gdf['latitude'] = temp_centroids.y
lakes_gdf['longitude'] = temp_centroids.x

# 3. Drop the 'centroid' column if it exists as an object
if 'centroid' in lakes_gdf.columns:
    lakes_gdf = lakes_gdf.drop(columns=['centroid'])

# 4. Create the map
Map = geemap.Map(center=[12.9716, 77.5946], zoom=11)
Map.add_basemap('HYBRID')

# 5. Add the first 50 lakes
# geemap handles the 'geometry' column automatically
Map.add_gdf(lakes_gdf.head(50), layer_name='geometry', info_mode='on_click', style={
        'color': '#00FFFF',   # Bright Cyan outline for visibility
        'width': 2,           # Thickness of the line
        'fill_opacity': 0     # Makes the center transparent
    })
Map.addLayer(
ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(ee.Geometry.Point([77.5946, 12.9716])).first(),
{'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 3000},
'Vegetation (Red) vs Water (Black)'
)

Map

In [ ]:
import pandas as pd
import ee
import geemap

def get_lake_area(lake_row, start_year=2020, end_year=2025):
    """
    Calculates water, weed, and total area for a specific lake over a time range.
    """
    results = []
    
    # Extract metadata from the row
    lake_name = lake_row['name']
    # Centroid for coordinates
    lon = lake_row.geometry.centroid.x
    lat = lake_row.geometry.centroid.y
    
    # Convert to Earth Engine Object
    # Using the original geometry for the strict weed mask
    lake_geom_ee = geemap.gdf_to_ee(gpd.GeoDataFrame([lake_row], crs=lakes_gdf.crs))
    
    # Create search zones
    # We use a 50m buffer for water (to allow for slight expansion)
    # But we clip weeds strictly to the 50m buffer to avoid counting distant trees
    search_zone = lake_geom_ee.geometry().buffer(50) 
    
    for year in range(start_year, end_year + 1):
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        scale = 10 if year >= 2017 else 30
        
        # 1. Collection Selection
        if year >= 2017:
            collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                .filterBounds(search_zone)
                .filterDate(start_date, end_date)
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                .select(['B3', 'B11', 'B8', 'B4'], ['Green', 'SWIR1', 'NIR', 'Red']))
        else:
            collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                .filterBounds(search_zone)
                .filterDate(start_date, end_date)
                .filter(ee.Filter.lt('CLOUD_COVER', 20))
                .select(['SR_B3', 'SR_B6', 'SR_B5', 'SR_B4'], ['Green', 'SWIR1', 'NIR', 'Red']))

        # 2. Check for data
        if collection.size().getInfo() == 0:
            results.append({
                'name': lake_name, 'lat': lat, 'lon': lon, 'year': year, 
                'open_water_m2': 0, 'weed_cover_m2': 0, 'total_area_m2': 0, 'status': 'no_data'
            })
            continue

        # 3. Create Median and Indices
        image = collection.median()
        mndwi = image.normalizedDifference(['Green', 'SWIR1']).rename('MNDWI')
        ndvi = image.normalizedDifference(['NIR', 'Red']).rename('NDVI')
        
        # 4. Create Binary Masks
        water_mask = mndwi.gt(0)
        # Refined weed mask: High vegetation AND not deep dry land (MNDWI > -0.5)
        weed_mask = ndvi.gt(0.4).And(mndwi.gt(-0.5))
        combined_mask = water_mask.Or(weed_mask).rename('Total_Area')
        
        # 5. Helper function for area reduction
        def run_stats(mask, band_name):
            area_img = mask.multiply(ee.Image.pixelArea())
            stats = area_img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=search_zone,
                scale=scale,
                maxPixels=1e9
            )
            return stats.getInfo().get(band_name) or 0

        # Execute calculations
        water_val = run_stats(water_mask, 'MNDWI')
        weed_val = run_stats(weed_mask, 'NDVI')
        total_val = run_stats(combined_mask, 'Total_Area')
        
        results.append({
            'name': lake_name,
            'lat': lat,
            'lon': lon,
            'year': year, 
            'open_water_m2': water_val,
            'weed_cover_m2': weed_val,
            'total_area_m2': total_val,
            'status': 'success'
        })
        print(f"{lake_name} ({year}): {round(total_val/10000, 2)} hectares")

    return pd.DataFrame(results)

In [ ]:
for idx in range(len(lakes_gdf)):
    get_lake_area(lakes_gdf.iloc[idx])

In [10]:
def get_lake_area_fast(lake_row, start_year=2020, end_year=2025):
    lake_name = lake_row['name']
    print(f"processing {lake_name}...")
    lon, lat = lake_row.geometry.centroid.x, lake_row.geometry.centroid.y
    
    # Pre-process geometry
    lake_geom_ee = geemap.gdf_to_ee(gpd.GeoDataFrame([lake_row], crs=lakes_gdf.crs))
    search_zone = lake_geom_ee.geometry().buffer(50)
    
    years = ee.List.sequence(start_year, end_year)

    def calculate_annual_stats(year):
        year = ee.Number(year)
        start_date = ee.Date.fromYMD(year, 1, 1)
        end_date = ee.Date.fromYMD(year, 12, 31)
        
        # Select collection based on year
        # Note: We use an ee.Algorithms.If to handle the Sentinel vs Landsat switch
        s2_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(search_zone)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                  .select(['B3', 'B11', 'B8', 'B4'], ['Green', 'SWIR1', 'NIR', 'Red']))
        
        l8_col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                  .filterBounds(search_zone)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUD_COVER', 20))
                  .select(['SR_B3', 'SR_B6', 'SR_B5', 'SR_B4'], ['Green', 'SWIR1', 'NIR', 'Red']))
        
        collection = ee.ImageCollection(ee.Algorithms.If(year.gte(2017), s2_col, l8_col))
        scale = ee.Number(ee.Algorithms.If(year.gte(2017), 10, 30))
        
        image = collection.median()
        
        # Indices
        mndwi = image.normalizedDifference(['Green', 'SWIR1']).rename('water')
        ndvi = image.normalizedDifference(['NIR', 'Red']).rename('weed')
        
        # Masks
        water_mask = mndwi.gt(0)
        weed_mask = ndvi.gt(0.4).And(mndwi.gt(-0.5))
        combined_mask = water_mask.Or(weed_mask).rename('total')
        
        # Combine masks into one 3-band image for a SINGLE reduction
        stats_img = ee.Image.cat([water_mask, weed_mask, combined_mask]).multiply(ee.Image.pixelArea())
        
        stats = stats_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=search_zone,
            scale=scale,
            maxPixels=1e9
        )

        # 1. Get the raw values from the dictionary
        # 2. Cast them to ee.Number so we can use .divide()
        water_m2 = ee.Number(stats.get('water'))
        weed_m2 = ee.Number(stats.get('weed'))
        total_m2 = ee.Number(stats.get('total'))
        
        # Return a Feature with the results in properties
        return ee.Feature(None, {
            'name': lake_name,
            'lat': lat,
            'lon': lon,
            'year': year,
            'open_water_ha': water_m2.divide(10000),
            'weed_cover_ha': weed_m2.divide(10000),
            'total_area_ha': total_m2.divide(10000)
        })

    # Run the map on the server
    annual_features = ee.FeatureCollection(years.map(calculate_annual_stats))
    
    # SINGLE .getInfo() call at the very end
    dict_results = annual_features.getInfo()['features']
    
    # Flatten and return
    return pd.DataFrame([f['properties'] for f in dict_results])

* To compare the static boundary (strictly inside the polygon) with the dynamic boundary (the 50m buffer), we can run two separate reduceRegion calls inside the same function.

* This is incredibly useful for Bengaluru lakes because if the buffer area is significantly larger than the static area, it often indicates flooding or, more likely, floating weeds and water expanding into areas that should legally be dry land or protected buffer zones.


In [16]:
def get_lake_area_fast(lake_row, start_year=2020, end_year=2025):
    lake_name = lake_row['name']
    print(f"processing {lake_name}...")
    lon, lat = lake_row.geometry.centroid.x, lake_row.geometry.centroid.y
    
    # Pre-process geometries
    lake_geom_ee = geemap.gdf_to_ee(gpd.GeoDataFrame([lake_row], crs=lakes_gdf.crs))
    strict_boundary = lake_geom_ee.geometry()
    buffered_boundary = strict_boundary.buffer(50)
    
    years = ee.List.sequence(start_year, end_year)

    def calculate_annual_stats(year):
        year = ee.Number(year)
        # Using Post-Monsoon filter for better accuracy as discussed
        s2_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(buffered_boundary)
                  .filter(ee.Filter.calendarRange(year, year, 'year'))
                  .filter(ee.Filter.calendarRange(11, 2, 'month')) 
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                  .select(['B3', 'B11', 'B8', 'B4'], ['Green', 'SWIR1', 'NIR', 'Red']))
        
        l8_col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                  .filterBounds(buffered_boundary)
                  .filter(ee.Filter.calendarRange(year, year, 'year'))
                  .filter(ee.Filter.calendarRange(11, 2, 'month'))
                  .filter(ee.Filter.lt('CLOUD_COVER', 20))
                  .select(['SR_B3', 'SR_B6', 'SR_B5', 'SR_B4'], ['Green', 'SWIR1', 'NIR', 'Red']))
        
        collection = ee.ImageCollection(ee.Algorithms.If(year.gte(2017), s2_col, l8_col))
        scale = ee.Number(ee.Algorithms.If(year.gte(2017), 10, 30))
        
        image = collection.median()
        
        # Indices
        mndwi = image.normalizedDifference(['Green', 'SWIR1']).rename('water')
        ndvi = image.normalizedDifference(['NIR', 'Red']).rename('weed')
        
        # Masks
        water_mask = mndwi.gt(0)
        weed_mask = ndvi.gt(0.4).And(mndwi.gt(-0.5))
        combined_mask = water_mask.Or(weed_mask).rename('total')
        
        # Area image
        area_img = ee.Image.cat([water_mask, weed_mask, combined_mask]).multiply(ee.Image.pixelArea())
        
        # REDUCTION 1: Static Boundary
        stats_static = area_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=strict_boundary,
            scale=scale,
            maxPixels=1e9
        )
        
        # REDUCTION 2: Buffered Boundary (50m)
        stats_buffer = area_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=buffered_boundary,
            scale=scale,
            maxPixels=1e9
        )

        # Helper to get numbers safely
        def get_ha(stats, key):
            return ee.Number(stats.get(key, 0)).divide(10000)

        return ee.Feature(None, {
            'name': lake_name,
            'year': year,
            'lat': lat,
            'lon': lon,
            # Static Results
            'static_water_ha': get_ha(stats_static, 'water'),
            'static_weed_ha': get_ha(stats_static, 'weed'),
            'static_total_ha': get_ha(stats_static, 'total'),
            # Buffered Results
            'buffer_water_ha': get_ha(stats_buffer, 'water'),
            'buffer_weed_ha': get_ha(stats_buffer, 'weed'),
            'buffer_total_ha': get_ha(stats_buffer, 'total')
        })

    annual_features = ee.FeatureCollection(years.map(calculate_annual_stats))
    return pd.DataFrame([f['properties'] for f in annual_features.getInfo()['features']])

### Capacity (Potential Area)
* To calculate the Capacity (Potential Area), we need to find the geometric area of the lake boundary polygon itself, regardless of what the satellite sees.
* While lakes_gdf has a geometry, calculating it within Earth Engine ensures that the "Potential Area" uses the exact same coordinate projection and geometric math as your "Water Area" and "Weed Area," making the comparison mathematically perfect.
* I have added a `potential_area_ha` calculation by reducing a constant "1" value over the lake geometry.
* Why this Comparison is needed:
    * By having `potential_ha`, I can answer two vital questions: 
        * **Utilization Rate**: How much of the official lake body is actually filled with water?
            * `df['water_fill_rate'] = df['static_water_ha'] / df['potential_ha']`
        * **Encroachment/Expansion**: Is the "Total Occupied Area" (Water + Weeds) exceeding the legal capacity?
            * `df['overflow_ratio'] = df['static_total_ha'] / df['potential_ha']`
        * **Note**: If this is **> 1.0**, the lake is physically larger than its legal survey (spilling over or surveyed incorrectly).

In [6]:
def get_lake_area_fast(lake_row, start_year=2020, end_year=2025):
    lake_name = lake_row['name']
    print(f"processing {lake_name}...")
    lon, lat = lake_row.geometry.centroid.x, lake_row.geometry.centroid.y
    
    # Pre-process geometries
    lake_geom_ee = geemap.gdf_to_ee(gpd.GeoDataFrame([lake_row], crs=lakes_gdf.crs))
    strict_boundary = lake_geom_ee.geometry()
    buffered_boundary = strict_boundary.buffer(50)
    
    # CALCULATE POTENTIAL CAPACITY (Static geometric area of the polygon)
    # We use a pixelArea of 1 multiplied by the reducer to get the total polygon size in m2
    potential_area_m2 = ee.Image.pixelArea().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=strict_boundary,
        scale=10, # Sentinel resolution for precision
        maxPixels=1e9
    ).get('area')
    
    # Convert to ha on the server side
    potential_ha = ee.Number(potential_area_m2).divide(10000)
    
    years = ee.List.sequence(start_year, end_year)

    def calculate_annual_stats(year):
        year = ee.Number(year)
        
        # Select collection based on year
        s2_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(buffered_boundary)
                  .filter(ee.Filter.calendarRange(year, year, 'year'))
                  .filter(ee.Filter.calendarRange(11, 2, 'month')) 
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                  .select(['B3', 'B11', 'B8', 'B4'], ['Green', 'SWIR1', 'NIR', 'Red']))
        
        l8_col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                  .filterBounds(buffered_boundary)
                  .filter(ee.Filter.calendarRange(year, year, 'year'))
                  .filter(ee.Filter.calendarRange(11, 2, 'month'))
                  .filter(ee.Filter.lt('CLOUD_COVER', 20))
                  .select(['SR_B3', 'SR_B6', 'SR_B5', 'SR_B4'], ['Green', 'SWIR1', 'NIR', 'Red']))
        
        collection = ee.ImageCollection(ee.Algorithms.If(year.gte(2017), s2_col, l8_col))
        scale = ee.Number(ee.Algorithms.If(year.gte(2017), 10, 30))
        
        image = collection.median()
        
        # Indices
        mndwi = image.normalizedDifference(['Green', 'SWIR1']).rename('water')
        ndvi = image.normalizedDifference(['NIR', 'Red']).rename('weed')
        
        # Masks
        water_mask = mndwi.gt(0)
        weed_mask = ndvi.gt(0.4).And(mndwi.gt(-0.5))
        combined_mask = water_mask.Or(weed_mask).rename('total')
        
        # Area image
        area_img = ee.Image.cat([water_mask, weed_mask, combined_mask]).multiply(ee.Image.pixelArea())
        
        # REDUCTION 1: Static Boundary
        stats_static = area_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=strict_boundary,
            scale=scale,
            maxPixels=1e9
        )
        
        # REDUCTION 2: Buffered Boundary (50m)
        stats_buffer = area_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=buffered_boundary,
            scale=scale,
            maxPixels=1e9
        )

        # Helper to get numbers safely
        def get_ha(stats, key):
            return ee.Number(stats.get(key, 0)).divide(10000)

        return ee.Feature(None, {
            'name': lake_name,
            'year': year,
            'lat': lat,
            'lon': lon,
            'potential_ha': potential_ha, # This stays constant for all years of a lake
            # Static Results
            'static_water_ha': get_ha(stats_static, 'water'),
            'static_weed_ha': get_ha(stats_static, 'weed'),
            'static_total_ha': get_ha(stats_static, 'total'),
            # Buffered Results
            'buffer_water_ha': get_ha(stats_buffer, 'water'),
            'buffer_weed_ha': get_ha(stats_buffer, 'weed'),
            'buffer_total_ha': get_ha(stats_buffer, 'total')
        })

    annual_features = ee.FeatureCollection(years.map(calculate_annual_stats))
    return pd.DataFrame([f['properties'] for f in annual_features.getInfo()['features']])

In [7]:
import os

# 1. Ensure the output directory exists
output_dir = 'data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 2. List to store the results of each lake
all_lakes_results = []

print(f"Starting batch processing for {len(lakes_gdf)} lakes...")

# 3. Loop through all 202 lakes in lakes_gdf
for idx in range(len(lakes_gdf)):
    lake_row = lakes_gdf.iloc[idx]
    
    try:
        # Call the fast server-side function
        # Using 2020-2025 as the default range
        df_lake = get_lake_area_fast(lake_row, start_year=2020, end_year=2025)
        
        # Add to our collection
        all_lakes_results.append(df_lake)
        
    except Exception as e:
        print(f"Error processing {lake_row['name']} (Index {idx}): {e}")

# 4. Concatenate all individual DataFrames into one master table
master_df = pd.concat(all_lakes_results, ignore_index=True)

# 5. Save the final result to the data folder
csv_filename = os.path.join(output_dir, 'bengaluru_lakes_master_2020_2025.csv')
master_df.to_csv(csv_filename, index=False)

print("-" * 30)
print(f"Processing Complete!")
print(f"Total rows generated: {len(master_df)}")
print(f"File saved to: {csv_filename}")

Starting batch processing for 202 lakes...
processing NCBS Pond...
processing Vengayyana Lake...
processing Halasuru lake...
processing Chelekere...
processing Madiwala Lake...
processing Iblur Lake...
processing Benniganahalli Lake...
processing Puttenhalli Lake...
processing Mathikere Lake...
processing Anchepalya Lake...
processing Sankey Tank...
processing Rachenahalli Lake...
processing Chinnappanahalli Lake...
processing Herohalli Kere...
processing Kaikondrahalli Lake...
processing Lal Bahadur Shastri Nagar Lake...
processing Agara Lake...
processing Hebbal Lake...
processing Sarakki Kere...
processing Jakkur Lake...
processing Yediyur Lake...
processing Seegehalli Lake...
processing Yelahanka Lake...
processing Hemmigepura Kere...
processing Thubarahalli Lake...
processing Agrahara Lake...
processing Puttenahalli Lake...
processing Attur Lake...
processing Allalasandra Lake...
processing Kodigehalli Lake...
processing Narsipura Lake...
processing Dasarahalli Tank...
processing 

In [18]:
df = pd.read_csv('data/bengaluru_lakes_master_2020_2025.csv')
df = df[df['potential_ha'] > 0.5]
df = df[['name', 'year', 'potential_ha', 'static_total_ha', 'buffer_total_ha']]
df = df.sort_values(['potential_ha', 'year'], ascending=False)
df.head()

,name,year,potential_ha,static_total_ha,buffer_total_ha
353,Bellandur Lake,2025,315.408905,232.051580,263.454482
352,Bellandur Lake,2024,315.408905,138.975866,153.664383
351,Bellandur Lake,2023,315.408905,138.511384,161.035938
350,Bellandur Lake,2022,315.408905,196.892749,220.268858
349,Bellandur Lake,2021,315.408905,155.451036,174.516059


### Lake Buffer & Land Cover Analysis Methodology
This analysis employs satellite remote sensing and multi-temporal geospatial processing via **Google Earth Engine (GEE)** to monitor the ecological health and urban encroachment of Bengaluru's lake systems.    
1. **Spatial Zoning Strategy**
    * The study utilizes a **"Donut Buffer"** approach to analyze land cover changes across two critical transition zones relative to the official lake boundary (Cadastral Boundary):
        * **Zone A (Riparian/Internal Zone)**: The area extending **50 meters inward** from the official boundary. This zone is critical for biodiversity and helps identify if the lake’s edges are being filled or if natural vegetation is present.
        * **Zone B (Encroachment/External Zone)**: The area extending **50 meters outward** from the official boundary. This represents the legally protected buffer zone in Bengaluru. Increasing "Built-up" area here signifies high encroachment pressure.
2. **Spectral Indices & Classification**
    * To differentiate between water, vegetation, and man-made structures, the following indices are calculated using **Sentinel-2 (10m resolution)** and **Landsat 8 (30m resolution)** imagery:\
        \
        **A. NDVI (Normalized Difference Vegetation Index)**: Used to identify Green Cover. It exploits the fact that healthy vegetation reflects high amounts of **Near-Infrared (NIR)** light.
        $$NDVI = \frac{NIR - Red}{NIR + Red}$$
            * Threshold: Values **> 0.4** are classified as dense green cover (trees/bushes).
        **B. NDBI (Normalized Difference Built-Up Index)**: Used to identify Buildings and Paved Surfaces. Urban areas have higher reflectance in the **Shortwave Infrared (SWIR)** range than the NIR range.
        $$NDBI = \frac{SWIR - NIR}{SWIR + NIR}$$
            * Threshold: Values $> 0$ (combined with low NDVI) are classified as built-up area.

        

In [19]:
def get_land_cover_stats(lake_row, start_year=2020, end_year=2025):
    lake_name = lake_row['name']
    print(f"processing {lake_name}...")
    
    # 1. Setup Geometries
    lake_gdf_single = gpd.GeoDataFrame([lake_row], crs=lakes_gdf.crs)
    lake_geom_ee = geemap.gdf_to_ee(lake_gdf_single)
    strict_boundary = lake_geom_ee.geometry()
    
    # Create the Donut Zones
    outer_buffer = strict_boundary.buffer(50)
    inner_buffer = strict_boundary.buffer(-50)
    
    # Zone A: 50m Inside the boundary (The Shoreline/Riparian zone)
    inside_50m_zone = strict_boundary.difference(inner_buffer)
    
    # Zone B: 50m Outside the boundary (The Neighborhood/Encroachment zone)
    outside_50m_zone = outer_buffer.difference(strict_boundary)
    
    years = ee.List.sequence(start_year, end_year)

    def calculate_annual_land_cover(year):
        year = ee.Number(year)
        
        # We use Sentinel-2 for better resolution (10m) on buildings
        collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                      .filterBounds(outer_buffer)
                      .filter(ee.Filter.calendarRange(year, year, 'year'))
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                      .median())
        
        # NDVI for Green Cover (NIR - Red) / (NIR + Red)
        ndvi = collection.normalizedDifference(['B8', 'B4']).rename('ndvi')
        
        # NDBI for Buildings (SWIR - NIR) / (SWIR + NIR)
        ndbi = collection.normalizedDifference(['B11', 'B8']).rename('ndbi')
        
        # Classification Thresholds
        green_mask = ndvi.gt(0.4).rename('green')
        building_mask = ndbi.gt(0.0).And(ndvi.lt(0.2)).rename('buildings')
        
        stats_img = ee.Image.cat([green_mask, building_mask]).multiply(ee.Image.pixelArea())
        
        # Helper to reduce by zone
        def get_stats(zone_geom):
            return stats_img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=zone_geom,
                scale=10,
                maxPixels=1e9
            )

        res_inside = get_stats(inside_50m_zone)
        res_outside = get_stats(outside_50m_zone)

        return ee.Feature(None, {
            'name': lake_name, 'year': year,
            'in_green_ha': ee.Number(res_inside.get('green', 0)).divide(10000),
            'in_build_ha': ee.Number(res_inside.get('buildings', 0)).divide(10000),
            'out_green_ha': ee.Number(res_outside.get('green', 0)).divide(10000),
            'out_build_ha': ee.Number(res_outside.get('buildings', 0)).divide(10000)
        })

    annual_fc = ee.FeatureCollection(years.map(calculate_annual_land_cover))
    return pd.DataFrame([f['properties'] for f in annual_fc.getInfo()['features']])

In [ ]:
all_land_results = []

for idx in range(len(lakes_gdf)):
    lake_row = lakes_gdf.iloc[idx]
    try:
        df_land = get_land_cover_stats(lake_row)
        all_land_results.append(df_land)
    except Exception as e:
        print(f"Skipped {lake_row['name']}: {e}")

# Save the final Land Cover dataset
if all_land_results:
    master_land_df = pd.concat(all_land_results, ignore_index=True)
    master_land_df.to_csv('data/bengaluru_lakes_land_cover.csv', index=False)

### Merging Datasets

In [ ]:
# Load the hydrological master data
df_master = pd.read_csv('data/bengaluru_lakes_master_2020_2025.csv')

# Load the land cover (buildings/greenery) data
df_land = pd.read_csv('data/bengaluru_lakes_land_cover.csv')

# Merge on common keys: 'name' and 'year'
df_combined = pd.merge(df_master, df_land, on=['name', 'year'], how='inner')

# Save the final consolidated dataset
df_combined.to_csv('data/bengaluru_lakes_combined_data_2020_2025.csv', index=False)

df = pd.read_csv('data/bengaluru_lakes_combined_data_2020_2025.csv')

# Remove duplicates based on name and year, keeping the first entry found
df_cleaned = df.drop_duplicates(subset=['name', 'year'], keep='first')

# Save the cleaned version
df_cleaned.to_csv('data/bengaluru_lakes.csv', index=False)

print(f"Cleaned dataset saved. Remaining rows: {len(df_cleaned)}")

Cleaned dataset saved. Remaining rows: 1086


In [8]:
df = pd.read_csv('data/bengaluru_lakes.csv')
df = df[['name', 'lat', 'lon', 'year', 'potential_ha', 
         'static_total_ha', 'static_water_ha', 'static_weed_ha',
         'buffer_total_ha', 'buffer_water_ha', 'buffer_weed_ha',
         'in_build_ha', 'out_build_ha', 'in_green_ha', 'out_green_ha']]
df = df.sort_values(['potential_ha', 'year'], ascending=False)
df = df[df['potential_ha'] > 0.5]

#df.to_csv('data/bengaluru_lakes_cleaned_gt_0.5ha.csv')

df.head()


,name,lat,lon,year,potential_ha,static_total_ha,static_water_ha,static_weed_ha,buffer_total_ha,buffer_water_ha,buffer_weed_ha,in_build_ha,out_build_ha,in_green_ha,out_green_ha
353,Bellandur Lake,12.934815,77.663797,2025,315.408905,232.051580,68.665089,169.082748,263.454482,68.665089,200.485650,0.337354,1.419036,70.471806,48.048649
352,Bellandur Lake,12.934815,77.663797,2024,315.408905,138.975866,61.476832,78.584039,153.664383,61.476832,93.272556,11.446859,9.832252,32.449680,20.454907
351,Bellandur Lake,12.934815,77.663797,2023,315.408905,138.511384,57.278417,118.409877,161.035938,57.299159,140.917298,7.953656,1.901681,40.736898,36.911006
350,Bellandur Lake,12.934815,77.663797,2022,315.408905,196.892749,26.795521,173.536269,220.268858,26.812654,196.895245,3.396715,2.887322,48.402764,35.146445
349,Bellandur Lake,12.934815,77.663797,2021,315.408905,155.451036,3.332483,155.315412,174.516059,3.332483,174.380434,9.962147,4.091976,45.100194,35.943240


In [20]:
df_mean = df.groupby(['name', 'lat', 'lon']).mean(numeric_only=True).reset_index()
df_mean = df_mean.sort_values(by='in_build_ha', ascending=False)
df_mean = df_mean.drop(columns=['year'])
df_mean['encroachment_pct'] = (df_mean['in_build_ha'] / df_mean['potential_ha']) * 100
df_mean = df_mean.sort_values(by='encroachment_pct', ascending=False)

df_mean.to_csv('data/bengaluru_lakes_mean.csv')
df_mean.head()

,name,lat,lon,potential_ha,static_total_ha,static_water_ha,static_weed_ha,buffer_total_ha,buffer_water_ha,buffer_weed_ha,in_build_ha,out_build_ha,in_green_ha,out_green_ha,encroachment_pct
32,Chikkabettahalli Lake,13.091797,77.554721,1.863068,0.521525,0.495941,0.025583,0.912038,0.616391,0.295648,1.522310,2.524853,0.025976,0.290142,81.709831
119,Panathur Chikka Kere,12.931470,77.708637,0.904825,0.473972,0.469141,0.006389,0.852702,0.485996,0.368320,0.732429,0.989031,0.002957,0.509506,80.947091
35,Chokkanahalli Lake,13.084613,77.629221,2.022320,0.455899,0.414647,0.041252,1.028474,0.417931,0.610543,1.527827,0.951821,0.054326,1.146548,75.548209
120,Panathuru Kere,12.931616,77.706855,4.149420,2.738649,2.690679,0.052478,4.448214,2.743214,1.711458,2.840596,1.198002,0.034464,2.042395,68.457661
95,Krishna Nagara Lake,12.874035,77.579749,4.421779,2.770499,2.670207,0.100292,4.383426,2.693766,1.689660,2.801031,0.944417,0.134707,3.034444,63.346238


In [7]:
import ee
import geemap

# 1. Setup
#Initialize Project

def map_lake_health(lake_name, start_year='2020', end_year='2025'):
    # Fetch actual irregular boundary from OSM
    try:
        lake_feature = geemap.osm_to_ee(f"{lake_name}, Bengaluru")
        lake_boundary = lake_feature.geometry()
    except:
        print(f"Boundary for {lake_name} not found. Check spelling.")
        return None

    # Load Sentinel-2 and clip to the JAGGED boundary
    s2_img = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(lake_boundary)
              .filterDate(f'{start_year}-01-01', f'{end_year}-12-31')
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
              .median()
              .clip(lake_boundary))

    # Calculate Indices
    mndwi = s2_img.normalizedDifference(['B3', 'B11']).rename('Water')
    ndvi = s2_img.normalizedDifference(['B8', 'B4']).rename('Vegetation')
    ndbi = s2_img.normalizedDifference(['B11', 'B8']).rename('Buildings')

    # NDTI (Turbidity/Silt Index) 
    # High values = Muddy water / Silt deposits
    ndti = s2_img.normalizedDifference(['B4', 'B3']).rename('Silt')

    # Create a mask to remove water and vegetation
    non_water_mask = mndwi.lt(0)  # Only areas that are NOT water
    non_veg_mask = ndvi.lt(0.2)   # Only areas that are NOT healthy plants
    brightness_mask = s2_img.select('B4').gt(0.3)

    # Calculate the refined Built-up Index
    refined_encroachment = (ndbi.updateMask(non_water_mask)
                            .updateMask(non_veg_mask)
                            .updateMask(brightness_mask))
    
    # 4. Refined Logic for Hardened Waste/Encroachment
    # Identifying non-water, non-veg, and high-reflectance (brightness) zones
    water_mask = mndwi.gt(0)
    
    # 5. AREA CALCULATION
    # Multiply the mask (1s and 0s) by pixel area in square meters
    area_sqm = water_mask.multiply(ee.Image.pixelArea())
    
    # Sum the area within the lake boundary
    stats = area_sqm.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=lake_boundary,
        scale=10, # Sentinel-2 resolution
        maxPixels=1e9
    )
    
    # Convert square meters to hectares
    water_area_sqm = ee.Number(stats.get('Water'))
    water_area_ha = water_area_sqm.divide(10000).getInfo()
    total_area_ha = lake_boundary.area().divide(10000).getInfo()

    print(f"--- Hydrological Analysis: {lake_name} ---")
    print(f"Total Legal Boundary Area: {total_area_ha:.2f} ha")
    print(f"Detected Static Water Area: {water_area_ha:.2f} ha")
    print(f"Water Coverage: {(water_area_ha/total_area_ha)*100:.2f}%")

    # Visualize
    Map = geemap.Map()
    Map.add_basemap('SATELLITE')
    Map.centerObject(lake_boundary, 17)
    
    # Add Layers
    Map.addLayer(ndbi, {'min': -0.1, 'max': 0.3, 'palette': ['white', 'red']}, 'Encroachment (Red)')
    Map.addLayer(refined_encroachment, {'min': 0, 'max': 0.4, 'palette': ['white', 'darkred']}, 'Refined Encroachment')
    Map.addLayer(ndti, {'min': -0.1, 'max': 0.2, 'palette': ['white', 'brown']}, '4. Silt/Turbidity (Brown)')
    Map.addLayer(ndvi, {'min': 0, 'max': 0.6, 'palette': ['white', 'green']}, 'Weeds (Green)')
    Map.addLayer(mndwi, {'min': -0.5, 'max': 0.2, 'palette': ['white', 'blue']}, 'Water (Blue)')
    Map.addLayer(lake_boundary, {'color': 'yellow'}, 'Legal Boundary')
    
    Map.add_layer_control()
    return Map

# --- EXECUTION ---
# Just change the name here to visualize any lake!
my_lake_map = map_lake_health("Pattandur Agrahara Lake")
my_lake_map

--- Hydrological Analysis: Pattandur Agrahara Lake ---
Total Legal Boundary Area: 4.54 ha
Detected Static Water Area: 0.00 ha
Water Coverage: 0.00%


Map(center=[12.976570168744892, 77.74581432872321], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap

def map_lake_truth_serum(lake_name, start_year='2023', end_year='2025'):
    # 1. Fetch Boundary from OSM
    try:
        lake_feature = geemap.osm_to_ee(f"{lake_name}, Bengaluru")
        lake_boundary = lake_feature.geometry()
    except:
        print(f"Boundary for {lake_name} not found.")
        return None

    # 2. Thermal Data (Landsat 8 Collection 2 Level 2)
    # This dataset provides Surface Temperature directly
    l8_img = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
              .filterBounds(lake_boundary)
              .filterDate(f'{start_year}-01-01', f'{end_year}-12-31')
              .filter(ee.Filter.lt('CLOUD_COVER', 10))
              .median())

    # Calculate LST in Celsius
    # Scale factor for C2 L2: 0.00341802 * DN + 149.0 (Kelvin)
    temp_c = l8_img.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15).clip(lake_boundary)

    # 3. Spectral Indices (Sentinel-2)
    s2_img = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(lake_boundary)
              .filterDate(f'{start_year}-01-01', f'{end_year}-12-31')
              .median()
              .clip(lake_boundary))

    mndwi = s2_img.normalizedDifference(['B3', 'B11']).rename('Water')
    ndvi = s2_img.normalizedDifference(['B8', 'B4']).rename('Vegetation')
    ndbi = s2_img.normalizedDifference(['B11', 'B8']).rename('NDBI')

    # 4. THE TRUTH SERUM LOGIC
    # Define "Hardened Waste" = High NDBI AND High Temperature (> 35°C)
    # This filters out "cool" dry silt that trick the NDBI
    #is_hot = temp_c.gt(10)
    #is_built = ndbi.gt(0.1)
    #not_water = mndwi.lt(0)

    # 1. Calculate the mean temperature of the whole lake area
    '''
    lake_mean_temp = temp_c.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=lake_boundary,
        scale=30
    ).get('ST_B10')
    

    # 2. Identify "Thermal Anomalies" (Pixels 2 degrees hotter than average)
    is_hot_anomaly = temp_c.gt(ee.Number(lake_mean_temp).add(2))
    '''
    # 1. Get the Temperature Stats for the Lake
    stats = temp_c.reduceRegion(
        reducer=ee.Reducer.percentile([50, 95]), # Get Median and Max
        geometry=lake_boundary,
        scale=30
    ).getInfo()

    median_temp = stats['ST_B10_p50']
    max_temp = stats['ST_B10_p95']

    # 2. Dynamic Truth Serum
    # Instead of > 35, we look for pixels 3 degrees hotter than the lake median
    is_hot_anomaly = temp_c.gt(median_temp + 1) 

    # 3. Refined Detection
    # Lower NDBI threshold slightly to 0.05 to catch smaller rubble piles
    confirmed_encroachment = ndbi.updateMask(is_hot_anomaly).updateMask(ndbi.gt(0.05))
    
    

    # 5. Visualization
    Map = geemap.Map()
    Map.centerObject(lake_boundary, 16)
    Map.add_basemap('SATELLITE')

    # Add Layers
    # Heat map: Blue (Cool) to Red (Hot)
    Map.addLayer(temp_c, {'min': 25, 'max': 45, 'palette': ['blue', 'yellow', 'orange', 'red']}, '1. Temperature (LST)')
    
    # Raw NDBI (The one that might have false positives)
    Map.addLayer(ndbi, {'min': -0.1, 'max': 0.3, 'palette': ['white', 'purple']}, '2. Raw NDBI Signature')
    
    # Confirmed Waste (The Truth)
    Map.addLayer(confirmed_encroachment, {'min': 0, 'max': 0.4, 'palette': ['white', 'darkred']}, '3. CONFIRMED Hardened Waste (Hot + Built)')
    
    # Boundary
    Map.addLayer(lake_boundary, {'color': 'yellow'}, 'Legal Boundary')

    return Map

# Execute for your lake
my_truth_map = map_lake_truth_serum("Chikkalasandra lake ")
my_truth_map